In [99]:
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten

In [100]:
!unzip 'BONE.zip'

Archive:  BONE.zip
replace BONE/Train/Avulsion fracture/12891_2019_2585_Fig5_HTML_png.rf.e7f1f0cb0b6723cde402057336779890.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [101]:
train_path='/content/BONE/Train'
validation_path='/content/BONE/Validation'

In [102]:
## NUMBER OF TRAIN_SAMPLE and TEST_SAMPLE
num_train_sample=587
num_validation_sample=73

In [103]:
epoch=34
batch_size=16


In [104]:
# Load the inceptation model without
base_model=InceptionV3(weights='imagenet',
include_top=False,
input_shape=(224,224,3)
)

In [105]:
## Freeze the model
for model in base_model.layers:
  model.trainable=False

In [106]:
model=Sequential()
model.add(base_model)

In [107]:
model.add(Flatten())
model.add(Dense(600, activation='relu'))
model.add(Dense(250, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(100, activation='relu'))
model.add(Dense(5, activation='softmax'))

In [108]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_v3 (Functional)   (None, 5, 5, 2048)        21802784  
                                                                 
 flatten_4 (Flatten)         (None, 51200)             0         
                                                                 
 dense_16 (Dense)            (None, 600)               30720600  
                                                                 
 dense_17 (Dense)            (None, 250)               150250    
                                                                 
 dropout_4 (Dropout)         (None, 250)               0         
                                                                 
 dense_18 (Dense)            (None, 100)               25100     
                                                                 
 dense_19 (Dense)            (None, 5)                

In [113]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [114]:
train_datagen=ImageDataGenerator(preprocessing_function=preprocess_input)
validation_datagen=ImageDataGenerator(preprocessing_function=preprocess_input)

In [115]:

train_generator= train_datagen.flow_from_directory(
    train_path,
    target_size=(224, 224),
    batch_size=batch_size,
)
validation_generator=validation_datagen.flow_from_directory(
    validation_path,
    target_size=(224,224),
    batch_size=batch_size,)

Found 587 images belonging to 5 classes.
Found 73 images belonging to 5 classes.


In [116]:
model.fit(train_generator,
          steps_per_epoch= num_train_sample // batch_size,
          epochs=epoch,
          validation_data= validation_generator,
          validation_steps=num_validation_sample // batch_size
          )

Epoch 1/34
36/36 [==============================] - 8s 98ms/step - loss: 10.4960 - accuracy: 0.2487 - val_loss: 3.9011 - val_accuracy: 0.3438
Epoch 2/34
36/36 [==============================] - 3s 72ms/step - loss: 3.1211 - accuracy: 0.3538 - val_loss: 1.9018 - val_accuracy: 0.4062
Epoch 3/34
36/36 [==============================] - 2s 59ms/step - loss: 1.4955 - accuracy: 0.5587 - val_loss: 1.6829 - val_accuracy: 0.5156
Epoch 4/34
36/36 [==============================] - 2s 57ms/step - loss: 0.9913 - accuracy: 0.6497 - val_loss: 1.5120 - val_accuracy: 0.5625
Epoch 5/34
36/36 [==============================] - 2s 57ms/step - loss: 0.5943 - accuracy: 0.7847 - val_loss: 2.3606 - val_accuracy: 0.4219
Epoch 6/34
36/36 [==============================] - 2s 59ms/step - loss: 0.4297 - accuracy: 0.8511 - val_loss: 1.9174 - val_accuracy: 0.5625
Epoch 7/34
36/36 [==============================] - 2s 65ms/step - loss: 0.2540 - accuracy: 0.9054 - val_loss: 2.5194 - val_accuracy: 0.5156
Epoch 8/34
3

In [117]:
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import os

In [118]:
model.save('Bone_fracture_model.h5')

In [119]:
model=load_model('/content/Bone_fracture_model.h5')

In [120]:
image_path="/content/X-ray-2-comminuted-displaced-fracture_png.rf.ace79e6f0ec1e6ec4037943cc83c14d4.jpg"

In [121]:
test_image=image.load_img(image_path, target_size=(224,224))
test_image=image.img_to_array(test_image)
test_image=np.expand_dims(test_image, axis=0)

In [122]:
np.argmax(model.predict(test_image), axis=1)

1/1 [==============================] - 1s 1s/step


array([4])